# Effect of UBI on Supplemental Poverty Measure

$500 per person.

## Setup
### Imports

In [1]:
import pandas as pd
import numpy as np
import microdf as mdf
import os

### Load data

Looks for the 2019 March Supplement in a `~/data` folder.

In [2]:
ASEC_F = '~/data/cpspb/asec/prod/data/2019/pppub19.csv'

In [3]:
if not os.path.isfile(os.path.expanduser(ASEC_F)):
    !mkdir ~/data
    !wget -O ~/data/asecpub19csv.zip http://thedataweb.rm.census.gov/pub/cps/march/asecpub19csv.zip
    !unzip ~/data/asecpub19csv.zip -d ~/data

In [4]:
SPM_COLS = ['povthreshold', 'resources', 'poor', 'numper', 'numkids', 'numadults', 'id']
OTHER_COLS = ['A_AGE', 'MARSUPWT', 'PRCITSHP']
cols = ['SPM_' + i.upper() for i in SPM_COLS] + OTHER_COLS

In [5]:
raw = pd.read_csv(ASEC_F, usecols=cols)

## Preprocess

In [6]:
df = raw.copy(deep=True)

In [7]:
df.columns = map(str.lower, df.columns)

Add true weight by dividing by 100.

In [8]:
df['w'] = df.marsupwt / 100.

Add citizenship indicator, per https://www2.census.gov/programs-surveys/cps/techdocs/cpsmar19.pdf.

In [9]:
df['is_citizen'] = df.prcitshp != 5

In [10]:
df['is_kid'] = df.a_age < 18
df['is_adult'] = df.a_age >= 18
df['is_kid_citizen'] = df.is_citizen & df.is_kid
df['is_adult_citizen'] = df.is_citizen & df.is_adult

Number of citizens per SPM unit.

In [11]:
spmu = df.groupby('spm_id')[['is_kid', 'is_adult', 'is_kid_citizen', 'is_adult_citizen']].sum()

In [12]:
spmu.columns = ['spm_nu18', 'spm_n18', 'spm_numkidcitizens', 'spm_numadultcitizens']

In [13]:
df = df.merge(spmu, on='spm_id')

## Analysis

Percent citizen.

In [14]:
mdf.weighted_mean(df, 'is_citizen', 'w')

0.9274755680739586

In [15]:
mdf.weighted_mean(df[df.a_age < 18], 'is_citizen', 'w')

0.9718902678491073

In [16]:
mdf.weighted_mean(df[df.a_age >= 18], 'is_citizen', 'w')

0.9143950894716393

In [17]:
def rounded_pct(x):
    print(str((x * 100).round(1)) + '%')

In [18]:
def print_pov(df, weight='w'):
    rounded_pct(mdf.weighted_mean(df, 'spm_poor', weight))

In [19]:
def ubi_pov(ubi_adult=0, ubi_kid=0, ubi_adult_citizen=0, ubi_kid_citizen=0):
    resources = (
        df.spm_resources + 
        ubi_adult * df.spm_n18 +
        ubi_kid * df.spm_nu18 +
        ubi_adult_citizen * df.spm_numadultcitizens +
        ubi_kid_citizen * df.spm_numkidcitizens)
    is_pov = resources < df.spm_povthreshold
    return (is_pov * df.w).sum() / df.w.sum()

In [20]:
def ubi_cost(ubi_adult=0, ubi_kid=0, ubi_adult_citizen=0, ubi_kid_citizen=0):
    ubi_adult_cost = (ubi_adult * df.is_adult * df.w).sum()
    ubi_kid_cost = (ubi_kid * df.is_kid * df.w).sum()
    ubi_adult_citizen_cost = (ubi_adult_citizen * df.is_adult_citizen * df.w).sum()
    ubi_kid_citizen_cost = (ubi_kid_citizen * df.is_kid_citizen * df.w).sum()
    return ubi_adult_cost + ubi_kid_cost + ubi_adult_citizen_cost + ubi_kid_citizen_cost

In [21]:
ubi_cost(6000, 6000) / 1e12

1.94613504762

In [22]:
ubi_cost(0, 0, 6000, 6000) / 1e12

1.80499270884

In [23]:
ubi_cost(7800, 0)  / 1e12

1.954391797566

In [24]:
ubi_pov(0, 0)

0.1273362169100547

In [25]:
ubi_pov(6000, 6000)

0.02927864490682862

In [26]:
ubi_pov(0, 0, 6000, 6000)

0.03907762854022117

In [27]:
1 - ubi_pov(6000, 6000) / ubi_pov(0, 0)

0.7700682051241564

In [28]:
ubi_pov(7800, 0)

0.031862851026619954

In [29]:
1 - ubi_pov(7800, 0) / ubi_pov(0, 0)

0.7497738522487548

In [30]:
ubi_cost(12000, 0) / 1e12

3.00675661164

## Miscellaneous

In [31]:
print_pov(df[df.is_citizen])

11.9%


In [32]:
print_pov(df[~df.is_citizen])

23.7%
